### Pip installs


In [ ]:
%pip install --upgrade bitsandbytes accelerate
%pip install peft
%pip install datasets
import locale # colab workaround
locale.getpreferredencoding = lambda: "UTF-8" 

### Loading Libs

In [ ]:
import os, sys, torch
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training 
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, BitsAndBytesConfig
from accelerate import Accelerator

### Load model and tokenizer from hugingface

In [ ]:
base_model = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    use_cache=False,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config = BitsAndBytesConfig(load_in_8bit=True)
)
tokenizer = AutoTokenizer.from_pretrained(base_model)

### Tokenization

In [ ]:

tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

### self-supervised learning

In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=2000,
        padding=False,
        return_tensors=None,
    )
    result["labels"] = result["input_ids"].copy()
    return result

### Prompt Generation

In [ ]:
def get_prompt(data_point):
    return f"""
### Instruction
Decompile the provided WAT snippet into an equivalent C code snippet, ensuring:
- Logical structure and functionality match the original WAT code.
- '<< >>' markers are preserved to indicate incomplete segments.
- Replace strings in the decompiled C code with their WAT data segment offsets. The format to represent these strings should be '(i32|i64.const offset)'.
- For variables in the decompiled C snippet, name them based on the value of the offset in wat snippet (i.e., i32.load offset=xxx), local variables are named local_{{offset}}.
- Even if the part of wat code is dead code, decompile it to c.

### Input(wat code):
The `Call_Func Declaration` specifies the number of params and return values.
[Call_Func Declaration]
{data_point['func_declaration']}
[/Call_Func Declaration]
The `Defined_Variable` tells the variables defined before current snippet.
[Defined_Variable]
{data_point['vars_has_defined']}
[/Defined_Variable]
[Wat]
{data_point["input"]}
[/Wat]

### Response(c code):
{data_point["output_replace_str"]}
"""

def generate_and_tokenize_prompt(data_point):
    full_prompt = get_prompt(data_point)
    return  tokenize(full_prompt)

### Load dataset

In [ ]:
from datasets import load_dataset, load_from_disk, Dataset
split_datasets = load_from_disk('../Dataset/split_dataset')
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

### Set Lora

In [ ]:
model.train()
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

### Auguments

In [ ]:
batch_size = 128
per_device_train_batch_size = 32
gradient_accumulation_steps = batch_size // per_device_train_batch_size
output_dir = "wasm2c"
training_args = TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
        max_steps=400,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_torch",
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=20,
        save_steps=20,
        output_dir=output_dir,
        load_best_model_at_end=False,
        group_by_length=True,
    )

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, 
        pad_to_multiple_of=8, 
        return_tensors="pt", 
        padding=True
    ),
)

In [ ]:
if torch.__version__ >= "2" and sys.platform != "win32":
    print("compiling the model")
    model = torch.compile(model)
trainer.train()